# 📓 The GenAI Revolution Cookbook

**Title:** How to Build Multi-Agent AI Systems with CrewAI and YAML

**Description:** Build production-ready multi-agent AI systems with CrewAI using reusable YAML-first patterns, explicit tools and tasks, guardrails, and interactive training loops.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## Introduction

This guide walks you through building a multi\-agent customer feedback analysis system using CrewAI. You will define three specialized agents for sentiment analysis, summarization, and visualization. They work together in a sequential workflow to process raw feedback, extract insights, then produce a final report with charts.

You will learn how to define agents and tasks in YAML, integrate tools—such as CSV search or file\-read—for grounded analysis, validate structured outputs, and execute code for visualization. All code is ready to run in Colab with minimal setup.

By the end, you will have a reproducible, auditable, multi\-agent system that turns customer feedback into actionable insights.

---

## Why Structure Matters with CrewAI

CrewAI gives you clear components—Agents, Tasks, Tools, Crews, and Processes—that align with how you build a team. Agents have roles, goals, and backstories. Tasks specify what to do and what to produce. Tools deliver concrete supports. Crews coordinate agents, and Processes define collaboration styles.

You will spend less time on repeated boilerplate. You will make your configurations versionable and readable by defining agents and tasks in YAML. That separation lets you tweak behaviors without touching core code.

You will also learn the value of grounding results in real data using CSV or file\-read tools. Grounding reduces hallucination and boosts trust. With memory features, agents can carry context forward.

If you’re interested in how these concepts compare to other agent frameworks, see our step\-by\-step guide on [building a stateful AI agent with LangGraph](https://thegenairevolution.com/blog/44830763/how-to-build-a-stateful-ai-agent-with-langgraph-step-by-step-5), which explores persistent memory and visual debugging.

---

## Core Concepts

**Agents**
Agents hold three core attributes: role (e.g., "Customer Feedback Analyst"), goal (what you want them to accomplish), and backstory (context guiding decisions). You define them in YAML so non\-technical reviewers can inspect or edit them. Agents can use tools and maintain memory.

**Tasks**
Tasks declare what needs doing. Each has a description, expected output format, and agent assignment. All in YAML. Tasks can run sequentially or in parallel depending on how you build your crew.

**Tools**
Tools supply extra capabilities. You might use a CSV search tool, a file\-reader, or web\-external APIs. Assign tools to agents or tasks. Agents use tools to avoid guessing—so output stays grounded and reliable.

For a deeper dive into building robust data extraction pipelines and minimizing hallucinations, check out our article on [structured data extraction with LLMs](https://thegenairevolution.com/blog/44830763/structured-data-extraction-with-llms-how-to-build-a-pipeline-3).

**Crews \& Processes**
A Crew combines agents and tasks into a workable system. Processes define how tasks are coordinated. Sequential workflows are simplest. But hierarchical, parallel, or hybrid processes serve more sophisticated use cases.

**Training and Feedback**
Your agents will not be perfect at first. Use CrewAI’s training mode to pause at interactions, give corrections, then let the agents adjust. This mirrors how you coach a new team member. Over time, performance improves.

---

## Step\-by\-Step Example: Customer Feedback Report

Here you build a feedback analyst system with three agents: for sentiment, summarization, and visualization. Then you build four tasks. Later you assemble and run it.

---

## Setup and Installation

In [ ]:
!pip install -q crewai==0.28.0 crewai-tools==0.1.6 pandas matplotlib pyyaml

Securely set your OpenAI API key in Colab:

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

print("API key set successfully.")

---

## Step 1: Sample Feedback Data

Create a sample CSV file if it does not already exist:

In [ ]:
import pandas as pd
from pathlib import Path

Path("figures").mkdir(exist_ok=True)
csv_path = Path("customer_feedback.csv")

if not csv_path.exists():
    data = [
        {"customer_id": 1, "feedback_text": "Loved the new UI, checkout was smooth!", "rating": 5, "date": "2024-09-01"},
        {"customer_id": 2, "feedback_text": "Support was slow. Still unresolved ticket.", "rating": 2, "date": "2024-09-03"},
        {"customer_id": 3, "feedback_text": "Great pricing but the app crashes sometimes.", "rating": 3, "date": "2024-09-05"},
        {"customer_id": 4, "feedback_text": "The UX is confusing on mobile.", "rating": 2, "date": "2024-09-08"},
        {"customer_id": 5, "feedback_text": "Fast delivery and friendly service!", "rating": 5, "date": "2024-09-12"},
    ]
    pd.DataFrame(data).to_csv(csv_path, index=False)
    print("Sample customer_feedback.csv created.")
else:
    print("customer_feedback.csv already exists.")

---

## Step 2: Define Agents in YAML

Write `agents.yaml` defining three agents:

```yaml
feedback_analysis_agent:
  role: "Customer Sentiment Analyst"
  goal: "Evaluate sentiment and themes from customer feedback accurately and reproducibly."
  backstory: >
    You analyze feedback at scale, balancing qualitative nuance with structured outputs.
    You use CSV search to reference concrete feedback rows. You avoid hallucinations.

summary_report_agent:
  role: "Insights Summarization Specialist"
  goal: "Produce an executive-ready summary table and concise narrative from analyzed feedback."
  backstory: >
    You write clearly, focus on business-relevant insights, and adhere to requested output formats.

visualization_agent:
  role: "Visualization Specialist"
  goal: "Generate executable Python code for sentiment distribution and trend charts."
  backstory: >
    You produce matplotlib code that reads customer_feedback.csv and saves figures to ./figures/.
```

---

## Step 3: Define Tasks in YAML

Write `tasks.yaml` with four tasks:

```yaml
sentiment_evaluation:
  description: >
    Read customer_feedback.csv. For each feedback row, infer sentiment ("positive", "neutral", or "negative"),
    and extract up to 3 themes (e.g. "pricing", "support", "ux"). Summarize counts per sentiment and
    top themes overall.
  expected_output: >
    JSON with keys:
    - "row_sentiments": list of {customer_id, date, sentiment, themes: [str]}
    - "sentiment_counts": {positive: int, neutral: int, negative: int}
    - "top_themes": list of {theme: str, count: int}

summary_table_creation:
  description: >
    Using sentiment evaluation JSON, create a concise summary table highlighting overall sentiment distribution,
    top 5 themes with counts, and 3 bullet insights for executives.
  expected_output: >
    Markdown containing:
    - "Sentiment Distribution" table with columns [sentiment, count]
    - "Top Themes" table with columns [theme, count]
    - "Key Insights" as 3 bullet points

chart_visualization:
  description: >
    Output executable Python/matplotlib code that reads customer_feedback.csv and saves
    figures to ./figures/sentiment_dist.png and ./figures/sentiment_trend.png.
  expected_output: >
    Python code block that generates and saves the two charts.

final_report_assembly:
  description: >
    Assemble the final report including: a brief narrative (<=150 words), the summary table markdown,
    and references to the generated figures. Ensure the report is clean and ready to share.
  expected_output: >
    Markdown with sections: "Overview", "Summary", "Figures", "Appendix: Method".
```

---

## Step 4: Load Configurations

Load both YAML files into Python:

In [ ]:
import yaml

def load_yaml(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

agents_cfg = load_yaml("agents.yaml")
tasks_cfg = load_yaml("tasks.yaml")
print("YAML configurations loaded.")

---

## Step 5: Instantiate Tools

Assign tools to agents. For feedback analysis, use a CSV search or file reader tool.

In [ ]:
from crewai_tools import CSVSearchTool

csv_tool = CSVSearchTool(csv="customer_feedback.csv")
print("CSVSearchTool instantiated.")

---

## Step 6: Create Agents in Code

Instantiate agents using configurations. Include tools and memory settings as needed.

In [ ]:
from crewai import Agent

feedback_analysis_agent = Agent(
    role=agents_cfg["feedback_analysis_agent"]["role"],
    goal=agents_cfg["feedback_analysis_agent"]["goal"],
    backstory=agents_cfg["feedback_analysis_agent"]["backstory"],
    tools=[csv_tool],
    memory=True
)

summary_report_agent = Agent(
    role=agents_cfg["summary_report_agent"]["role"],
    goal=agents_cfg["summary_report_agent"]["goal"],
    backstory=agents_cfg["summary_report_agent"]["backstory"],
    memory=True
)

visualization_agent = Agent(
    role=agents_cfg["visualization_agent"]["role"],
    goal=agents_cfg["visualization_agent"]["goal"],
    backstory=agents_cfg["visualization_agent"]["backstory"],
    memory=False
)

print("Agents created.")

---

## Step 7: Instantiate Tasks

Create Task instances assigned to agents:

In [ ]:
from crewai import Task

sentiment_evaluation = Task(
    description=tasks_cfg["sentiment_evaluation"]["description"],
    expected_output=tasks_cfg["sentiment_evaluation"]["expected_output"],
    agent=feedback_analysis_agent
)

summary_table_creation = Task(
    description=tasks_cfg["summary_table_creation"]["description"],
    expected_output=tasks_cfg["summary_table_creation"]["expected_output"],
    agent=summary_report_agent
)

chart_visualization = Task(
    description=tasks_cfg["chart_visualization"]["description"],
    expected_output=tasks_cfg["chart_visualization"]["expected_output"],
    agent=visualization_agent
)

final_report_assembly = Task(
    description=tasks_cfg["final_report_assembly"]["description"],
    expected_output=tasks_cfg["final_report_assembly"]["expected_output"],
    agent=summary_report_agent
)

print("Tasks created.")

---

## Step 8: Assemble the Crew and Define Process

Build a Crew with your agents and tasks. Choose sequential execution so tasks follow in order.

In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[feedback_analysis_agent, summary_report_agent, visualization_agent],
    tasks=[sentiment_evaluation, summary_table_creation, chart_visualization, final_report_assembly],
    process=Process.sequential
)

print("Crew assembled.")

---

## Step 9: Run Your Workflow

Execute the multi\-agent workflow and capture the final report. Track execution time.

In [ ]:
import time

start = time.perf_counter()
result = crew.kickoff()
elapsed = time.perf_counter() - start

print("\n=== Final Report ===\n")
print(result)
print(f"\nElapsed: {elapsed:.2f}s")

---

## Run and Evaluate

### Validate Structured JSON Output

Confirm that sentiment evaluation output matches the expected schema.

In [ ]:
import json

def validate_sentiment_json(output: str) -> bool:
    try:
        analysis_json = json.loads(output)
        required_keys = {"row_sentiments", "sentiment_counts", "top_themes"}
        if not required_keys.issubset(analysis_json.keys()):
            print("Missing required keys in sentiment analysis output.")
            return False
        print("Sentiment JSON is valid.")
        return True
    except Exception as e:
        print(f"Analysis output is not valid JSON: {e}")
        return False

# Example usage:
# validate_sentiment_json(sentiment_evaluation.output)

### Execute Visualization Code

Run code snippets produced by the visualization agent to generate the charts.

In [ ]:
def run_plotting_snippet(snippet: str):
    namespace = {}
    try:
        exec(snippet, namespace, namespace)
        print("Plotting code executed successfully.")
    except Exception as e:
        print(f"Error executing plotting code: {e}")

# Example usage:
# if "plt." in chart_visualization.output:
#     run_plotting_snippet(chart_visualization.output)

### Verify Saved Figures

Check that expected figure files exist:

In [ ]:
import os

expected_figures = ["figures/sentiment_dist.png", "figures/sentiment_trend.png"]
for fig in expected_figures:
    if os.path.exists(fig):
        print(f"{fig} exists.")
    else:
        print(f"{fig} not found.")

---

## Conclusion and Next Steps

You have built a multi\-agent feedback analysis system with CrewAI. You defined agents and tasks in YAML. You grounded your analysis with CSV/data tools. You validated structured output formats. You ran visualization code. You executed everything in sequence for clarity and auditability.

To go further: introduce a quality assurance gate to check intermediate outputs before assembling the final report. Or parallelize tasks like summary and visualization to save time when dependencies allow. You might also explore other process types such as hierarchical or hybrid process modes to suit more complex workflows.

For a broader perspective on building LLM agents from scratch—including tool use and control loops—see our practical walkthrough on [building an LLM agent with GPT\-4 ReAct](https://thegenairevolution.com/blog/44830763/how-to-build-an-llm-agent-from-scratch-with-gpt-4-react-2).

For future reading: see The GenAI Revolution’s guide on [structured data extraction pipelines](https://thegenairevolution.com/blog/44830763/structured-data-extraction-with-llms-how-to-build-a-pipeline-3). You may also compare building agents from scratch using ReAct patterns or using frameworks like [LangGraph](https://thegenairevolution.com/blog/44830763/how-to-build-a-stateful-ai-agent-with-langgraph-step-by-step-5).

Begin!